In [1]:
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings('ignore')

In [2]:
spark = SparkSession.builder.master('local').appName('duplicate').getOrCreate()

21/07/14 04:14:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/07/14 04:14:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/07/14 04:14:43 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [35]:
dataF = spark.createDataFrame([('1', 2400, 32, '9', 3, 'PHILIPS', '1XZFG')
                                  , ('2', 2400, 23, '13', 3, 'Apple', '2XZFG')
                                  , ('3', 2400, 22, '2.5', 5, 'LG', '3XZFG')
                                  , ('4', 1, 23, '16.5', 3, None, '4XZFG')
                                  , ('8', 3500, 33, '23', 5, 'LG', '8XZFG')
                                  , ('6', 2400, 23, '9', 3, 'SONY', '6XZFG')
                                  , ('7', 3500, 34, '23', None, 'PHILIPS', '7XZFG')
                                  , ('8', 2500, 33, '23', 5, 'LG', '8XZFG')
                                  , ('9', 2400, 22, '2.5', 0, 'Apple', '9XZFG')
                                  , ('18', 2400, 22, '2.5', 0, 'Apple', '9XZFG')
                                  , ('10', None, 23, '6', 4, None, None)
                                  , ('15', 2750, 27, '8', 1, 'SONY', 'DCEZX15')
                                  , ('11', 2, 20, '6', 3, 'SONY', '11XZFG')
                                  , ('12', 3500, 32, '6', 3, 'LG', '12XZFG')
                                  , ('17', 3500, 39, '6', 3, 'LG', '12XZFG')
                                  , ('13', 2400, 24, '16.5', 5, 'Apple', '13XZFG')
                                  , ('14', 3500, 34, '6', 6, 'DELL', '14XZFG')
                                  , ('15', 2400, 22, '6', 4, 'SONY', '15XZFG')],
                              ['Id', 'Salary', 'age', 'experience', 'Coef', 'TV ', 'CODE_ID'])

In [36]:
features = ['Salary', 'age', 'Coef']

In [37]:
quantiles = [0.25, 0.75]

In [56]:
cutOffPoints = []

In [26]:
!pip install py

In [27]:
from py.path import local
from pyspark.sql.types import *
from pyspark.sql import Row as row
from pyspark.sql import DataFrameStatFunctions as statFunc

In [57]:
for feature in features : 
    qts = statFunc(dataF).approxQuantile(feature, quantiles, 0)
    IQR = qts[1] - qts[0]
    cutOffPoints.append((feature, [qts[0] - 1.5 * IQR, qts[1] + 1.5 * IQR]))
cutOffPoints = dict(cutOffPoints)

In [58]:
cutOffPoints

{'Salary': [750.0, 5150.0], 'age': [5.5, 49.5], 'Coef': [0.0, 8.0]}

In [59]:
aberrant_value = dataF.select(*['id'] + [
    (
        (dataF[f] < cutOffPoints[f][0]) |
        (dataF[f] > cutOffPoints[f][1])
    ).alias(f + '_b') for f in features
])

In [60]:
aberrant_value.show()

+---+--------+-----+------+
| id|Salary_b|age_b|Coef_b|
+---+--------+-----+------+
|  1|   false|false| false|
|  2|   false|false| false|
|  3|   false|false| false|
|  4|    true|false| false|
|  8|   false|false| false|
|  6|   false|false| false|
|  7|   false|false|  null|
|  8|   false|false| false|
|  9|   false|false| false|
| 18|   false|false| false|
| 10|    null|false| false|
| 15|   false|false| false|
| 11|    true|false| false|
| 12|   false|false| false|
| 17|   false|false| false|
| 13|   false|false| false|
| 14|   false|false| false|
| 15|   false|false| false|
+---+--------+-----+------+



In [61]:
withaberrant_value = dataF.join(aberrant_value, on = 'Id')

In [62]:
(
    withaberrant_value.filter('Salary_b').select("Id",'Salary','Coef').show()
)

+---+------+----+
| Id|Salary|Coef|
+---+------+----+
| 11|     2|   3|
|  4|     1|   3|
+---+------+----+



In [63]:
df_no_aberrent_value = (
    withaberrant_value.filter('!Salary_b').select(dataF.columns)
)

In [64]:
df_no_aberrent_value.show()

+---+------+---+----------+----+-------+-------+
| Id|Salary|age|experience|Coef|    TV |CODE_ID|
+---+------+---+----------+----+-------+-------+
|  7|  3500| 34|        23|null|PHILIPS|  7XZFG|
| 15|  2750| 27|         8|   1|   SONY|DCEZX15|
| 15|  2750| 27|         8|   1|   SONY|DCEZX15|
| 15|  2400| 22|         6|   4|   SONY| 15XZFG|
| 15|  2400| 22|         6|   4|   SONY| 15XZFG|
|  3|  2400| 22|       2.5|   5|     LG|  3XZFG|
|  8|  3500| 33|        23|   5|     LG|  8XZFG|
|  8|  3500| 33|        23|   5|     LG|  8XZFG|
|  8|  2500| 33|        23|   5|     LG|  8XZFG|
|  8|  2500| 33|        23|   5|     LG|  8XZFG|
| 18|  2400| 22|       2.5|   0|  Apple|  9XZFG|
| 17|  3500| 39|         6|   3|     LG| 12XZFG|
|  6|  2400| 23|         9|   3|   SONY|  6XZFG|
|  9|  2400| 22|       2.5|   0|  Apple|  9XZFG|
|  1|  2400| 32|         9|   3|PHILIPS|  1XZFG|
| 12|  3500| 32|         6|   3|     LG| 12XZFG|
| 13|  2400| 24|      16.5|   5|  Apple| 13XZFG|
| 14|  3500| 34|    

In [65]:
descriptive_stats = df_no_aberrent_value.describe()
descriptive_stats.show()

+-------+------------------+------------------+------------------+------------------+------------------+-----+-------+
|summary|                Id|            Salary|               age|        experience|              Coef|  TV |CODE_ID|
+-------+------------------+------------------+------------------+------------------+------------------+-----+-------+
|  count|                19|                19|                19|                19|                18|   19|     19|
|   mean|10.210526315789474|2794.7368421052633|28.263157894736842|11.368421052631579| 3.388888888888889| null|   null|
| stddev| 5.148667005140705| 503.5691326372469| 5.635621875013321| 7.886067364633158|1.8515141304249878| null|   null|
|    min|                 1|              2400|                22|                13|                 0|Apple| 12XZFG|
|    max|                 9|              3500|                39|                 9|                 6| SONY|DCEZX15|
+-------+------------------+------------------+-

In [66]:
descriptive_stats_f = df_no_aberrent_value.describe(features)
descriptive_stats_f.show()

+-------+------------------+------------------+------------------+
|summary|            Salary|               age|              Coef|
+-------+------------------+------------------+------------------+
|  count|                19|                19|                18|
|   mean|2794.7368421052633|28.263157894736842| 3.388888888888889|
| stddev| 503.5691326372469| 5.635621875013321|1.8515141304249878|
|    min|              2400|                22|                 0|
|    max|              3500|                39|                 6|
+-------+------------------+------------------+------------------+



In [69]:
correlation = df_no_aberrent_value.corr('Salary', 'age')
print('CORRELATION : ', correlation)

CORRELATION :  0.7639837160241238
